In [56]:
!nvidia-smi

Wed May 19 21:48:45 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  On   | 00000000:00:05.0 Off |                  Off |
| N/A   38C    P0    27W / 250W |     11MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
!conda info


     active environment : dl38
    active env location : /opt/conda/envs/dl38
            shell level : 1
       user config file : /opt/ml/.condarc
 populated config files : 
          conda version : 4.10.1
    conda-build version : 3.18.11
         python version : 3.7.7.final.0
       virtual packages : __cuda=10.1=0
                          __linux=4.4.0=0
                          __glibc=2.27=0
                          __unix=0=0
                          __archspec=1=x86_64
       base environment : /opt/conda  (writable)
      conda av data dir : /opt/conda/etc/conda
  conda av metadata url : https://repo.anaconda.com/pkgs/main
           channel URLs : https://repo.anaconda.com/pkgs/main/linux-64
                          https://repo.anaconda.com/pkgs/main/noarch
                          https://repo.anaconda.com/pkgs/r/linux-64
                          https://repo.anaconda.com/pkgs/r/noarch
          package cache : /opt/conda/pkgs
                          /opt/ml/.c

# KLUQUAD

KLUE + KORQUAD

In [3]:
import torch
from torch.utils.data import Dataset

import datasets
from datasets import load_from_disk, load_dataset

import pandas as pd
import numpy as np

from tqdm import tqdm

In [4]:
datasets.__version__

'1.6.2'

In [5]:
torch.cuda.is_available()

True

In [6]:
import os, sys
from pathlib import Path
from glob import glob

In [7]:
BASE_PATH = Path('.').resolve().parent
BASE_PATH

PosixPath('/opt/ml/jaepil')

In [8]:
sys.path.append(BASE_PATH.as_posix())

In [9]:
sys.path

['/opt/ml/jaepil/code',
 '/opt/conda/envs/dl38/lib/python38.zip',
 '/opt/conda/envs/dl38/lib/python3.8',
 '/opt/conda/envs/dl38/lib/python3.8/lib-dynload',
 '',
 '/opt/conda/envs/dl38/lib/python3.8/site-packages',
 '/opt/conda/envs/dl38/lib/python3.8/site-packages/huggingface_hub-0.0.8-py3.8.egg',
 '/opt/conda/envs/dl38/lib/python3.8/site-packages/locket-0.2.1-py3.8.egg',
 '/opt/conda/envs/dl38/lib/python3.8/site-packages/IPython/extensions',
 '/opt/ml/.ipython',
 '/opt/ml/jaepil']

## Load Data

In [10]:
data_path = BASE_PATH / "input" / "data" / "data"
data_path

PosixPath('/opt/ml/jaepil/input/data/data')

In [11]:
train_path = data_path / "train_dataset"

train_data_path = train_path / "train"
val_data_path = train_path / "validation"

In [12]:
test_path = data_path / "test_dataset"

test_data_path = test_path / "validation"

In [47]:
klue = load_from_disk(train_path)
klue

DatasetDict({
    train: Dataset({
        features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
        num_rows: 3952
    })
    validation: Dataset({
        features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
        num_rows: 240
    })
})

conda-forge 채널의 `datasets`는 1.2.1 이라 `remove_columns`가 안된다. 

huggingface 채널에서 깔아줘야 1.6.1 최신버전이 깔린다. 

하지만 아래에서 pandas로 concat할 것이라면 굳이 공통 column만 남겨줄 필요 없다. 

In [14]:
# klue_remove_columns = ['__index_level_0__', 'document_id', 'id',]

# klue = klue.remove_columns(klue_remove_columns) # requires higher datasets version
# klue

DatasetDict({
    train: Dataset({
        features: ['answers', 'context', 'question', 'title'],
        num_rows: 3952
    })
    validation: Dataset({
        features: ['answers', 'context', 'question', 'title'],
        num_rows: 240
    })
})

In [48]:
korquad = load_dataset("squad_kor_v1")
korquad

Reusing dataset squad_kor_v1 (/opt/ml/.cache/huggingface/datasets/squad_kor_v1/squad_kor_v1/1.0.0/92f88eedc7d67b3f38389e8682eabe68caa450442cc4f7370a27873dbc045fe4)


DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 60407
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 5774
    })
})

In [17]:
# korquad_remove_columns = ['id']
# korquad = korquad.remove_columns(korquad_remove_columns)
# korquad

DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'question', 'answers'],
        num_rows: 60407
    })
    validation: Dataset({
        features: ['title', 'context', 'question', 'answers'],
        num_rows: 5774
    })
})

In [39]:
korquad['train'].features

{'title': Value(dtype='string', id=None),
 'context': Value(dtype='string', id=None),
 'question': Value(dtype='string', id=None),
 'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None)}

## Concat data

그냥 두 `DatasetDict`를 concatenate 하려고 했지만 
```
    ValueError: Features must match for all datasets 
```
에러가 발생한다. 

그냥 팀원들 코드처럼 pandas 기반으로 하겠다. 

In [53]:
%%time

kluquad = datasets.DatasetDict()

for key in ['train', 'validation']:
    merged_df = pd.concat(
        [pd.DataFrame(klue[key]), pd.DataFrame(korquad[key])],
        join='inner',
        ignore_index=True,
    )
    kluquad[key] = datasets.Dataset.from_pandas(merged_df)

kluquad

DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers'],
        num_rows: 64359
    })
    validation: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers'],
        num_rows: 6014
    })
})

In [54]:
kluquad.save_to_disk(data_path / "kluquad")

In [55]:
kluquad = datasets.DatasetDict.load_from_disk(data_path / "kluquad")
kluquad

DatasetDict({
    train: Dataset({
        features: ['answers', 'context', 'id', 'question', 'title'],
        num_rows: 64359
    })
    validation: Dataset({
        features: ['answers', 'context', 'id', 'question', 'title'],
        num_rows: 6014
    })
})